In [ ]:
# This allows importing Jupyter notebooks as modules
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import JupyterNotebookImporter

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, LSTM
from keras.utils import to_categorical
import keras.backend as kb
import keras
from sklearn.metrics import mean_squared_error
import numpy as np
from keras import regularizers, optimizers, initializers
from numpy.random import seed
import AlgoPlotting as plt
import AlgoPrediction

In [ ]:
def shape_it(X):
    return np.expand_dims(X.reshape((-1,1)),2)

In [ ]:
# # Generate data
# data = np.random.randint(-5,5,200)

# n_data = len(data)
# data = np.matrix(data)
# n_train = int(0.8*n_data)

# index_delay = 5
# y_train = shape_it(data[:,:n_train])
# x_train = shape_it(data[:,index_delay:(n_train+index_delay)])
# y_test = shape_it(data[:,n_train:-index_delay])
# x_test = shape_it(data[:,(n_train+index_delay):])

In [ ]:
def generate_data():
    
    N = 200
    delay = 10
    #data = np.random.randint(-1, 2, N + delay)
    data = (np.random.rand(N + delay))
    data2 = (np.random.rand(N + delay))
    # data = np.cumsum(data)
    x = data[delay:]
    x2 = data2[delay:]
    #y = [data[ii]+data[ii+1]**2 + data[ii+2]**0.5 for ii in range(N)]
    y = [(data[ii]+data[ii+1])*x2[ii] for ii in range(N)]
    
#     x = []
#     for ii in range(n):
#         if np.random.rand() > 0.01 + 0.99 * ii/n:
#             x.append(-1)
#         else:
#             x.append(1)
    
#     y = x[1:]
#     x = x[0:-1]
        
#     print(x)
#     print(y)
#     print(np.shape(x))
#     print(np.shape(y))
    
    x = shape_it(np.array(x))
    x2 = shape_it(np.array(x2))
    #print(x.shape)
    x = np.hstack((x, x2))
    #print(x.shape)
    y = shape_it(np.array(y))
    #print(y.shape)
    return x, y
    
#generate_data()

In [ ]:
# Plot training data
# training_chart = plt.XYChart(y=[x_train, y_train],
#                              names=['X Train','Y Train'],
#                              title='Training Data'
#                             )

In [ ]:
# Plot testing data
# testing_chart = plt.XYChart(y=[x_test, y_test],
#                             names=['X Test','Y Test'],
#                             title='Testing Data'
#                            )

In [ ]:
class SGDLearningRateTracker(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        opt = self.model.optimizer
        
        # Not sure why but eval'ing 1 by 1 then calculating doesn't crash, but 1 line does
        lr = kb.eval(opt.lr)
        decay = kb.eval(opt.decay)
        iterations = kb.eval(opt.iterations)
        #lr = kb.eval(opt.lr * (1. / (1. + opt.decay * opt.iterations))) # This crashes
        lr = (lr * (1. / (1. + decay * iterations)))
        #print('\nLR: {:.6f}\n'.format(lr))
        logs['lr_calc'] = lr

In [ ]:
class PatternPredictingNetwork:
    def __init__(self, model_dict, x_train, y_train, x_test, y_test):
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test
        self.model = Sequential()
        self.batch_size = 1
        
        batch_input_shape=(self.batch_size, self.x_train.shape[1], self.x_train.shape[2])
        use_rnn_model = True
        
        if use_rnn_model:
            los = 'mse'
            act = 'tanh'
            learning_rate = 0.0001*9
            decay_rate = learning_rate/100
            momentum = 0.0
            sgd = optimizers.SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
            opt = sgd
            kreg = regularizers.l2(0.001)
            areg = regularizers.l1(0.001)
            neurons = 30
            num_recurrent_layers = 3
            for _ in range(max(num_recurrent_layers - 1, 0)):
                self.model.add(SimpleRNN(neurons, activation=act, batch_input_shape=batch_input_shape, stateful=True, return_sequences=True,
                                         kernel_regularizer=kreg, activity_regularizer=areg))
            self.model.add(SimpleRNN(neurons, activation=act, batch_input_shape=batch_input_shape, stateful=True,
                                     kernel_regularizer=kreg, activity_regularizer=areg))
            self.model.add(Dense(neurons, activation=act, 
                                 kernel_regularizer=kreg, activity_regularizer=areg))
            self.model.add(Dense(1))

            self.model.compile(loss=los, optimizer=opt)
        else:
            los = 'mse'
            act = 'tanh'
            learning_rate = 0.01*9
            decay_rate = learning_rate/100
            momentum = 0.0
            sgd = optimizers.SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
            opt = sgd
            kreg = regularizers.l2(0.0001)
            areg = regularizers.l2(0.0001)
            neurons = 30
            num_recurrent_layers = 2
            for _ in range(max(num_recurrent_layers - 1, 0)):
                self.model.add(LSTM(neurons, activation=act, batch_input_shape=batch_input_shape, stateful=True, return_sequences=True,
                                    kernel_regularizer=kreg, activity_regularizer=areg))
            self.model.add(LSTM(neurons, activation=act, batch_input_shape=batch_input_shape, stateful=True,
                                kernel_regularizer=kreg, activity_regularizer=areg))
            self.model.add(Dense(neurons, activation=act, 
                                 kernel_regularizer=kreg, activity_regularizer=areg))
            self.model.add(Dense(1))

            self.model.compile(loss=los, optimizer=opt)
        
        
    def train_and_monitor(self, epochs=100, epoch_update_interval=10, losses_trim=0, losses_window=None):
        # Initialize empty lists
        predictions = []
        losses = []
        learning_rates = []
        
        # Initialize charts
        predictions_chart = plt.XYChart(title='Predictions', x_label='Index', y_label='Y', names=['Prediction', 'Test Data'])
        losses_chart = plt.XYChart(title='Losses', x_label='Epochs', y_label='Loss')
        dlosses_chart = plt.XYChart(title='Diff(Losses)', x_label='Epochs', y_label='Diff(Loss)')
        lr_chart = plt.XYChart(title='Learning Rate', x_label='Epochs', y_label='Learning Rate')

        for epoch in range(epochs):
            self.x_train, self.y_train = generate_data()
            self.x_test, self.y_test = generate_data()
            #self.x_test = self.x_train
            #self.y_test = self.y_train
            
            #self.model.predict(shape_it(np.array(-1)), batch_size=batch_size)
            history = self.model.fit(self.x_train, np.reshape(self.y_train,(-1,)), epochs=1,
                                     batch_size=self.batch_size, verbose=0, shuffle=False, callbacks=[SGDLearningRateTracker()])
#             history = self.model.fit(self.x_train, self.y_train, epochs=1,
#                                      batch_size=batch_size, verbose=0, shuffle=False, callbacks=[SGDLearningRateTracker()])
            loss = history.history['loss'][0]
            lr = history.history['lr_calc'][0]
            losses.append(loss)
            learning_rates.append(lr)

            # Update the plots
            if not epoch % epoch_update_interval:
                self.model.reset_states()
                predictions = []
                predictions = self.model.predict(self.x_test, batch_size=self.batch_size, verbose=0)
                expected = self.y_test
                
                # Push off the starting indices once the losses is long enough
                if losses_window is not None:
                    if len(losses) <= losses_window:
                            y_losses = losses
                            x_losses = np.arange(len(y_losses))
                    else:
                        start = len(losses) - losses_window
                        y_losses = losses[start:]
                        x_losses = np.arange(start, len(losses))
                else:
                    if len(losses) <= losses_trim:
                        y_losses = losses
                        x_losses = np.arange(len(y_losses))
                    elif len(losses) > losses_trim and len(losses) < 2*losses_trim:
                        start = len(losses) - losses_trim
                        y_losses = losses[start:]
                        x_losses = np.arange(start, len(losses))
                    else:
                        y_losses = losses[losses_trim:]
                        x_losses = np.arange(losses_trim, len(losses))

                predictions_chart.update(y=[predictions, self.y_test])
                losses_chart.update(x=x_losses, y=y_losses)
                dlosses_chart.update(x=x_losses[:-1], y=np.diff(y_losses))
                lr_chart.update(y=learning_rates)
            self.model.reset_states()
    
    # Forecast with updates each step
    def one_step_forecast(self):
        batch_size = 1
        # One step forecast on testing data
        self.model.reset_states()
        self.model.predict(self.x_train, batch_size=batch_size)
        predictions = []
        for ii in range(len(self.x_test)):
            # make one-step forecast
            X = self.x_test[ii]
            X = X.reshape(1, 1, 1)
            yhat = self.model.predict(X, batch_size=batch_size)[0,0]

            # store forecast
            predictions.append(yhat)
            expected = self.y_test[ii]
            print('Index=%d, Predicted=%f, Expected=%f' % (ii, yhat, expected))

        # report performance
        rmse = np.sqrt(mean_squared_error(self.y_test.reshape(len(self.y_test)), predictions))
        print('Test RMSE: %.3f' % rmse)
        xy_chart = plt.XYChart(y=[predictions, self.y_test], names=['Prediction', 'Test Data'], title='One Step Prediction')
        
    def dynamic_forecast(self):
        batch_size = 1
        # Dynamic forecast on test data
        self.model.reset_states()
        self.model.predict(self.x_train, batch_size=batch_size)

        dynpredictions = list()
        dyhat = self.x_test[0]


        for ii in range(len(self.x_test)):
            # make one-step forecast
            dyhat = yhat.reshape(1, 1, 1)
            dyhat = model.predict(dyhat, batch_size=batch_size)[0,0]

            # store forecast
            dynpredictions.append(dyhat)
            expected = self.y_test[ii]
            print('Index=%d, Predicted Dynamically=%f, Expected=%f' % (ii, dyhat, expected))


        drmse = np.sqrt(mean_squared_error(Y_test.reshape(len(self.y_test)), dynpredictions))
        print('Test Dynamic RMSE: %.3f' % drmse)
        xy_chart = plt.XYChart(y=[dynpredictions, self.y_test], names=['Dynamic Prediction', 'Test Data'], title='Dynamic Prediction')
        

In [ ]:
model_dict = {}
epochs = 1000
epoch_update_interval = 10
x_train, y_train = generate_data()
x_test, y_test = generate_data()
pattern_predicting_network = PatternPredictingNetwork(model_dict, x_train, y_train, x_test, y_test)
# pattern_predicting_network.model.layers[0].set_weights(
#     #[np.array([[1.048831]]), np.array([[-0.05480201]])]
#     [np.array([[0.95]]), np.array([[0]])]
# )
# pattern_predicting_network.model.reset_states()
# pattern_predicting_network.model.predict(shape_it(np.array(-1)), batch_size=1)
pattern_predicting_network.train_and_monitor(epochs=epochs, epoch_update_interval=epoch_update_interval, losses_window=None)

In [ ]:
pattern_predicting_network.one_step_forecast()

In [ ]:
from __future__ import absolute_import
from __future__ import print_function

import keras
from keras import backend as K
import numpy as np


class LossLearningRateScheduler(keras.callbacks.History):
    """
    A learning rate scheduler that relies on changes in loss function
    value to dictate whether learning rate is decayed or not.
    LossLearningRateScheduler has the following properties:
    base_lr: the starting learning rate
    lookback_epochs: the number of epochs in the past to compare with the loss function at the current epoch to determine if progress is being made.
    decay_threshold / decay_multiple: if loss function has not improved by a factor of decay_threshold * lookback_epochs, then decay_multiple will be applied to the learning rate.
    spike_epochs: list of the epoch numbers where you want to spike the learning rate.
    spike_multiple: the multiple applied to the current learning rate for a spike.
    """

    def __init__(self, base_lr, lookback_epochs, spike_epochs = None, spike_multiple = 10, decay_threshold = 0.002, decay_multiple = 0.5, loss_type = 'val_loss'):

        super(LossLearningRateScheduler, self).__init__()

        self.base_lr = base_lr
        self.lookback_epochs = lookback_epochs
        self.spike_epochs = spike_epochs
        self.spike_multiple = spike_multiple
        self.decay_threshold = decay_threshold
        self.decay_multiple = decay_multiple
        self.loss_type = loss_type


    def on_epoch_begin(self, epoch, logs=None):
        
        if len(self.epoch) > self.lookback_epochs:

            current_lr = K.get_value(self.model.optimizer.lr)

            target_loss = self.history[self.loss_type] 

            loss_diff =  target_loss[-int(self.lookback_epochs)] - target_loss[-1]

            if loss_diff <= np.abs(target_loss[-1]) * (self.decay_threshold * self.lookback_epochs):

                print(' '.join(('Changing learning rate from', str(current_lr), 'to', str(current_lr * self.decay_multiple))))
                K.set_value(self.model.optimizer.lr, current_lr * self.decay_multiple)
                current_lr = current_lr * self.decay_multiple

            else:

                print(' '.join(('Learning rate:', str(current_lr))))

            if self.spike_epochs is not None and len(self.epoch) in self.spike_epochs:
                print(' '.join(('Spiking learning rate from', str(current_lr), 'to', str(current_lr * self.spike_multiple))))
                K.set_value(self.model.optimizer.lr, current_lr * self.spike_multiple)

        else:

            print(' '.join(('Setting learning rate to', str(self.base_lr))))
            K.set_value(self.model.optimizer.lr, self.base_lr)


        return K.get_value(self.model.optimizer.lr)




def main():
    return

if __name__ == '__main__':
    main()
